# Clase Práctica #1 (Compilación)

A lo largo del curso estaremos implementando un compilador para el lenguaje de programación `HULK` (*Havana University Language for Kompilers*), paso a paso, introduciendo nuevas características del lenguaje o mejorando la implementación de otras características a medida que vamos descubriendo las técnicas fundamentales de la teoría de lenguajes y la compilación.

El objetivo de esta clase es construir un evaluador de expresiones "a mano", usando los recursos que tenemos hasta el momento. Para ello vamos a comenzar con una versión de `HULK` muy sencilla, un lenguaje de expresiones aritméticas.

## Evaluador de expresiones

Definiremos a continuación este lenguaje de manera informal:

Un programa en `HULK` consta de una secuencia de expresiones. Cada expresión está compuesta por:

- números (con coma flotante de 32 bits), 
- operadores `+ *` con el orden operacional, y
- paréntesis `(` y `)`

### Análisis lexicográfico

Comenzaremos construyendo un prototipo bien simple, donde asumiremos que en la expresión hay espacios en blanco entre todos los elementos, de modo que el *lexer* se reduce a dividir por espacios. Luego iremos adicionando elementos más complejos.

El siguiente método devuelve una lista de *tokens*, asumiendo que la expresión solo tiene números, operadores y paréntesis, separados por espacios en blanco.

In [25]:
def tokenize(text):
    """
    Returns the set of tokens. At this point, simply splits by 
    spaces and converts numbers to `float` instances.
    """
    tokens = []
    for item in text.split():
        if item.isnumeric():
            tokens.append(float(item))
        else:
            tokens.append(item)
        
    return tokens

assert tokenize('5 + 6 * 9') == [5, '+', 6, '*', 9]

### Análisis sintáctico y evaluación

Una vez que tenemos los *tokens*, solo nos queda evaluar la expresión. Usaremos para ello una idea simple, pero bien útil: evaluaremos recursivamente la expresión descendiendo por los distintos niveles de precedencia.

Toda expresión del lenguaje puede ser vista como una suma de _términos_, donde cada uno de estos "_términos_" se descompone a su vez en operaciones de multiplicación entre _factores_. Incluso si no hay operadores `+` en toda la expresión queda claro que esta idea es válida puesto que estaríamos en presencia de una expresión formada por un solo _término_. Los _factores_ del lenguaje son todos unidades atómicas: por ahora solo números y expresiones complejas envueltas entre paréntesis. Nótese que el uso de paréntesis permite reiniciar el descenso por los niveles de precedencia (regresar a los niveles más altos).

In [26]:
# These lambda expressions map from operators to actual executable code
operations = {
    '+': lambda x,y: x + y,
    '*': lambda x,y: x * y,
}

In [27]:
# Some util classes and methods

class ParsingError(Exception):
    """
    Base class for all parsing exceptions.
    """
    pass

class BadEOFError(ParsingError):
    """
    Unexpected EOF error.
    """
    
    def __init__(self):
        ParsingError.__init__(self, "Unexpected EOF")
        
class UnexpectedToken(ParsingError):
    """
    Unexpected token error.
    """
    
    def __init__(self, token, i):
        ParsingError.__init__(self, f'Unexpected token: {token} at {i}')
        
class MissingCloseParenthesisError(ParsingError):
    """
    Missing ')' token error.
    """
    
    def __init__(self, token, i):
        ParsingError.__init__(self, f'Expected ")" token at {i}. Got "{token}" instead')
        
class MissingOpenParenthesisError(ParsingError):
    """
    Missing '(' token error.
    """
    
    def __init__(self, token, i):
        ParsingError.__init__(self, f'Expected "(" token at {i}. Got "{token}" instead')

def get_token(tokens, i, error_type=BadEOFError):
    """
    Returns tokens[i] if 'i' is in range. Otherwise, raises ParsingError exception.
    """
    try:
        return tokens[i]
    except IndexError:
        raise error_type()

In [28]:
def evaluate(tokens):
    """
    Evaluates an expression recursively.
    """
    try:
        i, value = parse_expression(tokens, 0)
        assert i == len(tokens)
        return value
    except ParsingError as error:
        print(error)
        return None

def parse_expression(tokens, i):
    i, term = parse_term(tokens, i)
    return parse_group_of_terms(tokens, i, term)
    
def parse_group_of_terms(tokens, i, value):
    if i < len(tokens):
        if get_token(tokens, i) == '+':
            i, term = parse_term(tokens, i + 1)
            value = value + term
            i, value = parse_group_of_terms(tokens, i, value)
        elif get_token(tokens, i) == '-':
            i, term = parse_term(tokens, i + 1)
            value = value - term
            i, value = parse_group_of_terms(tokens, i, value)
    return i, value

def parse_term(tokens, i):
    i, factor = parse_factor(tokens, i)
    return parse_group_of_factors(tokens, i, factor)

def parse_group_of_factors(tokens, i, value):
    if i < len(tokens):
        if get_token(tokens, i) == '*':
            i, factor = parse_factor(tokens, i + 1)
            value = value * factor
            i, value = parse_group_of_factors(tokens, i, value)
        elif get_token(tokens, i) == '/':
            i, factor = parse_factor(tokens, i + 1)
            value = value / factor
            i, value = parse_group_of_factors(tokens, i, value)
    return i, value

def parse_factor(tokens, i):
    token = get_token(tokens, i)
    if i < len(tokens):
        if token == '(':
            i, expr = parse_expression(tokens, i + 1)
            if get_token(tokens, i) != ')':
                raise MissingCloseParenthesisError(token, i)
            return i + 1, expr 
        elif token == ')':
            raise MissingOpenParenthesisError(token, i)
    return i + 1, float(get_token(tokens, i))
            
assert evaluate(tokenize('5 + 6 * 9')) == 59.
assert evaluate(tokenize('( 5 + 6 ) * 9')) == 99.
assert evaluate(tokenize('( 5 + 6 ) + 1 * 9 + 2')) == 22.
assert evaluate(tokenize('2 * 3 * 4 + 1')) == 25.

## Adicionando constantes

Agreguemos constantes numéricas al lenguaje `HULK` Para ello, simplemente añadiremos un diccionario con todas las constantes disponibles, que usaremos durante la tokenización. Nótese que solo es necesario modificar el _lexer_ para añadir este rasgo al lenguaje.

In [29]:
constants = {
    'pi': 3.14159265359,
    'e': 2.71828182846,
    'phi': 1.61803398875,
}

In [30]:
def tokenize(expr):
    """
    Returns the set of tokens. At this point, simply splits by 
    spaces and converts numbers to `float` instances.
    Replaces constants.
    """
    tokens = []
    
    for token in expr.split():
        if token.isnumeric():
            tokens.append(float(token))
        elif token in constants:
            tokens.append(float(constants[token]))
        else:
            tokens.append(token)
    
    return tokens

assert tokenize('2 * pi') == [2.0, '*', 3.14159265359]
assert evaluate(tokenize('2 * pi')) == 6.28318530718

## Adicionando operadores `-` y `/`

- **Restricción:** No utilizar ciclos!!!

In [31]:
# These lambda expressions map from operators to actual executable code
operations = {
    '+': lambda x,y: x + y,
    '-': lambda x,y: x - y,
    '*': lambda x,y: x * y,
    '/': lambda x,y: x / y,
}

In [32]:
# def parse_expression(tokens, i):
#     # Insert your code here ...
#     pass

        
# def parse_term(tokens, i):
#     # Insert your code here ...
#     pass



In [33]:
assert evaluate(tokenize('1 - 1 + 1')) == 1
assert evaluate(tokenize('8 / 4 / 2')) == 1

## Adicionando funciones elementales

Agreguemos funciones elementales `sin`, `cos`, `tan`, `log`, `sqrt`, etc. El llamado a funciones se hará en notación prefija, comenzando por el nombre de la función y seguido, entre paréntesis, por los argumentos, que estarán separados entre sí por _comas_.

Para las funciones elementales haremos algo similar a las constantes, pero en vez de a la hora de tokenizar, las reemplazaremos a la hora de evaluar, pues necesitamos evaluar recursivamente los argumentos de la función. Empezaremos por garantizar que nuestro tokenizador que es capaz de reconocer expresiones con funciones elementales de más de un argumento, en caso de no ser así es necesario arreglarlo.

In [34]:
assert tokenize('log ( 64 , 1 + 3 )') == ['log', '(', 64.0, ',', 1.0, '+', 3.0, ')']

Adicionaremos entonces un diccionario con todas las funciones elementales que permitiremos.

In [35]:
import math

functions = {
    'sin': lambda x: math.sin(x),
    'cos': lambda x: math.cos(x),
    'tan': lambda x: math.tan(x),
    'log': lambda x,y: math.log(x, y),
    'sqrt': lambda x: math.sqrt(x),
}

Por último, modificaremos el método `evaluate` para que use las funciones elementales. Recordemos que los argumentos están separados por el token _coma_ (`,`) y que cada uno puede a su vez tener sub-expresiones que consistan también en llamados a funciones.

In [38]:
def parse_factor(tokens, i):
    if i < len(tokens):
        if get_token(tokens, i) == '(':
            i, expr = parse_expression(tokens, i + 1)
            if get_token(tokens, i) != ')':
                raise MissingCloseParenthesisError(get_token(tokens, i), i)
            return i + 1, expr

        elif get_token(tokens, i) == ')':
            raise MissingOpenParenthesisError(get_token(tokens, i), i)

        elif get_token(tokens, i) == 'sin':
            return one_parameter_function('sin', tokens, i)

        elif get_token(tokens, i) == 'cos':
            return one_parameter_function('cos', tokens, i)

        elif get_token(tokens, i) == 'tan':
            return one_parameter_function('tan', tokens, i)

        elif get_token(tokens, i) == 'log':
            return two_parameter_function('log', tokens, i)

        elif get_token(tokens, i) == 'sqrt':
            return one_parameter_function('sqrt', tokens, i)

    return i + 1, get_token(tokens, i)

def one_parameter_function(function, tokens, i):
    if get_token(tokens, i + 1) != '(':
        raise MissingOpenParenthesisError(tokens, i)
    i, expr = parse_expression(tokens, i + 2)
    expr = functions[function](expr)
    if get_token(tokens, i) != ')':
                raise MissingCloseParenthesisError(tokens, i)
    return i + 1, expr

def two_parameter_function(function, tokens, i):
    if get_token(tokens, i + 1) != '(':
        raise MissingOpenParenthesisError(tokens, i)
    if get_token(tokens, i + 3) != ',':
        raise UnexpectedToken(tokens, i)
    i, expr1 = parse_expression(tokens, i + 2)
    i, expr2 = parse_expression(tokens, i + 1)
    expr = functions['log'](expr1, expr2)
    if get_token(tokens, i) != ')':
        raise MissingCloseParenthesisError(tokens, i)
    return i + 1, expr
    
assert evaluate(tokenize('log ( 64 , 1 + 3 )')) == 3.0